# Prompting LLMs for Patient-Oriented Language 

In [1]:
import os
import transformers
import json

import torch
import tqdm

from typing import Dict, List, Optional

from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, TextStreamer
from datasets import load_dataset


## Load dataset

In [2]:
data = load_dataset("csv", data_files="pleaseee.csv")

Found cached dataset csv (/linkhome/rech/genlig01/ulj12fo/.cache/huggingface/datasets/csv/default-c10b8bc08219fdf5/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|##########| 1/1 [00:00<00:00, 598.08it/s]


In [3]:
#data = load_dataset("Mathilde/test_data_pol")
subset = data['train']  # TODO changer en test dans le ds original
print(subset)
"""
statements_pol = data['train']['statement_pol']
statements_med = data['train']['statement_med']
premises =  data['train']['premise']
"""

Dataset({
    features: ['id', 'topic_id', 'statement_medical', 'statement_pol', 'premise', 'NCT_title', 'NCT_id', 'label'],
    num_rows: 1578
})


"\nstatements_pol = data['train']['statement_pol']\nstatements_med = data['train']['statement_med']\npremises =  data['train']['premise']\n"

## Load Model

TODO: change the model's path

In [4]:
model_path = "../../baselines/Qwen2.5-7B-Instruct" # "/gpfsdswork/dataset/HuggingFace_Models/meta-llama/Llama-2-7b-chat-hf" #"/lustre/fsn1/projects/rech/hjp/ulj12fo/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)

<class 'OSError'>: Incorrect path_or_model_id: '../baselines/Qwen2.5-7B-Instruct'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [5]:
streamer = TextStreamer(tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True)

<class 'NameError'>: name 'tokenizer' is not defined

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_path, load_in_8bit=True, device_map="auto",)  # torch_dtype=torch.float16, low_cpu_mem_usage=True

## Format the prompt

In [ ]:
def get_input_text(premise, hypothesis):
    # TODO the persona option + the wrapping of the pol + med "patients with this medical profile..."
    options_prefix = "Answer only with: \n- " #"OPTIONS:\n- "
    separator = "\n- "
    options_ = options_prefix + f"{separator}".join(["Entailment","Contradiction"])  # , "Neutral"
    return f"{premise} \n Question: Does the previous eligibility criteria imply that the following patient can participate to the trial?\n {hypothesis}\n {options_}"
    #return f"{premise} \n Question: Does this imply that {hypothesis}? {options_}"
    # Does the previous eligibility criteria imply that the following patient can participate to the trial?
    # return f"Classification: {premise} \n Question: Does this imply that {hypothesis}? Entailment or Contradiction?Answer:"

In [ ]:
subset[14]

In [ ]:
samples = []
# TODO adapt in function of the med or POL (l.5)
for instance in subset:
    premise = instance['premise']
    sentence = f"Eligibility criteria of the trial are:\n {premise}"
    input_text = get_input_text(sentence, instance['statement_pol'])
    # temp = {"text":input_text, "label":sample['label']}
    temp = {"text":input_text, "label":0}
    print(input_text)
    samples.append(temp)

## Define the chat function

TODO: adjust the hyperparameters

In [ ]:
def chat(
    query: str,
    history: Optional[List[Dict]] = None,
    temperature: float = 0.7,
    top_p: float = 1.0,
    top_k: float = 0,
    repetition_penalty: float = 1.1,
    max_new_tokens: int = 5, # 1024,
    **kwargs,
):
    if history is None:
        history = []

    history.append({"role": "user", "content": query})

    input_ids = tokenizer.apply_chat_template(history, add_generation_prompt=True, return_tensors="pt").to(model.device)
    input_length = input_ids.shape[1]

    generated_outputs = model.generate(
        input_ids=input_ids,
        generation_config=GenerationConfig(
            temperature=temperature,
            do_sample=temperature > 0.0,
            top_p=top_p,
            top_k=top_k,
            repetition_penalty=repetition_penalty,
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.eos_token_id,
            **kwargs,
        ),
        streamer=streamer,
        return_dict_in_generate=True,
    )

    generated_tokens = generated_outputs.sequences[0, input_length:]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    history.append({"role": "assistant", "content": generated_text})

    return generated_text, history

In [ ]:
response, history = chat(samples[52]['text'], history=None)

In [ ]:
samples[52]['text']

## Call the chat function on the whole test set

In [ ]:
labels = []
pred = []
#with torch.inference_mode():
for sample in tqdm.tqdm(samples):
    labels.append(sample["label"])
    # input_ids = tokenizer(sample["text"], return_tensors="pt",).input_ids.to("cuda")
    # input_ids = tokenizer.apply_chat_template(sample["text"], return_tensors="pt",).to("cuda")
    # outputs = model.generate(input_ids, max_new_tokens=20)
    response, history = chat(sample['text'], history=None)
    pred.append(response)

In [ ]:
pred

In [ ]:
# pred = [p[5:][:-4].strip() for p in pred]

## Parse model's output

TODO: adapt the pattern in function of the model's output style

In [ ]:
pattern = "entailment|contradiction|Entailment|Contradiction|entail"  # |Neutral|neutral

In [ ]:
import re

# Function to match text with regular expression
def match_text_with_regexp(text, pattern):
    text = text.strip()
    # print(text)
    # Compile the regular expression pattern
    regexp = re.compile(pattern)
    
    # Search for a match in the text
    match = regexp.search(text)
    # print(match)
    
    if match:
        # If a match is found, return the matched text
        return match.group()
    else:
        # If no match is found
        return None

parsed_preds = []

for p in pred:
    # text = pred[0]
    pattern = "entailment|contradiction|Entailment|Contradiction|entail"  # |Neutral|neutral
    
    result = match_text_with_regexp(p, pattern)
    
    if result:
        if result in ['entailment', 'entail', 'yes', 'Yes']:
            result = 'Entailment'
        elif result in ['contradiction', 'contradicts', 'no', 'No']:
            result = 'Contradiction'
        #elif result in ['neutral', 'Neutral']:
            #result = 'Neutral'
        # print(f"Match found: {result}")
        parsed_preds.append(result) 
    else:
        # print("No match found.")
        # if nothing exploitable predicted --> assert "Neutral"
        parsed_preds.append("Contradiction")
        


In [ ]:
parsed_preds

In [ ]:
set(parsed_preds)
from collections import Counter
Counter(parsed_preds)

## Save the predictions in a JSON file

In [ ]:
prediction_dict = {}
for _id,pred_x in zip(data['train']['id'], parsed_preds):
    prediction_dict[str(_id)] = {"Prediction":pred_x}

In [ ]:
json.dump(prediction_dict, open("results_qwen_2-5_7B_zs_pol.json",'w'),indent=4)